# LangWatch Evaluation Tracking

## Simple Evaluation Loop

In [1]:
import langwatch

langwatch.login()

Please go to https://app.langwatch.ai/authorize to get your API key
2025-05-26 11:53:44,848 - langwatch.client - INFO - Registering atexit handler to flush tracer provider on exit
LangWatch API key set


In [1]:
import random
import pandas as pd
import time

df = pd.DataFrame(
    [
        {
            "question": "What is LangWatch?",
            "answer": "LangWatch is a platform for evaluating and improving language models.",
        },
        {
            "question": "How do I use LangWatch?",
            "answer": "You can use LangWatch by installing the LangWatch SDK and then calling the LangWatch API.",
        },
        {
            "question": "Does LangWatch support multiple language models?",
            "answer": "Yes, LangWatch is compatible with all language models by using LiteLLM under the hood.",
        },
        {
            "question": "Can I visualize evaluation metrics in LangWatch?",
            "answer": "Yes, LangWatch provides dashboards for visualizing key evaluation metrics.",
        },
        {
            "question": "Is there a free tier for LangWatch?",
            "answer": "LangWatch offers a free tier with limited usage, ideal for small projects and evaluation.",
        },
        {
            "question": "Where can I find documentation for LangWatch?",
            "answer": "You can find the official documentation on the LangWatch website or GitHub repository.",
        },
    ]
)

evaluation = langwatch.evaluation.init("my-incredible-experiment")


@langwatch.trace()
def agent(question):
    time.sleep(random.randint(0, 10))
    return {"text": "foo bar"}


for index, row in evaluation.loop(df.iterrows()):
    result = agent(row["question"])  # your code

    score = random.randint(0, 80) / 100 + 0.2
    evaluation.log("sample_metric", index=index, score=score, passed=score > 0.5)


2025-05-26 11:31:23,761 - langwatch.utils.initialization - INFO - Setting up LangWatch client...
2025-05-26 11:31:23,773 - langwatch.client - INFO - Configuring OTLP exporter with endpoint: https://app.langwatch.ai/api/otel/v1/traces
2025-05-26 11:31:23,776 - langwatch.client - INFO - Registering atexit handler to flush tracer provider on exit
2025-05-26 11:31:23,778 - langwatch.client - INFO - Successfully configured tracer provider with OTLP exporter
2025-05-26 11:31:23,781 - langwatch.utils.initialization - INFO - LangWatch client setup complete
Follow the results at: https://app.langwatch.ai/inbox-narrator/experiments/my-incredible-experiment?runId=xanthic-hyrax-of-tenacity


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

## Parallel Evaluation Loop

In [3]:
import random
import time

langwatch.setup()
evaluation = langwatch.evaluation.init("my-incredible-experiment")

@langwatch.trace()
def agent(question):
    time.sleep(random.randint(0, 10))
    return "foo parallel"

for index, row in evaluation.loop(df.iterrows(), threads=4):
    def evaluate(index, row):
        result = agent(row["question"])
        evaluation.log("sample_metric", index=index, data={"response": result}, score=1)
    evaluation.submit(evaluate, index, row)

2025-05-26 11:32:11,315 - langwatch.client - INFO - Registering atexit handler to flush tracer provider on exit
2025-05-26 11:32:11,315 - langwatch.client - WARNING - An existing global trace provider was found. LangWatch will not override it automatically, but instead is attaching another span processor and exporter to it. You can disable this warning by setting `ignore_global_tracer_provider_override_warning` to `True`.
Follow the results at: https://app.langwatch.ai/inbox-narrator/experiments/my-incredible-experiment?runId=eager-sawfish-of-intensity


Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

Failed to detach context
Traceback (most recent call last):
  File "/Users/rchaves/Projects/langwatch-saas/langwatch/python-sdk/.venv/lib/python3.11/site-packages/opentelemetry/context/__init__.py", line 155, in detach
    _RUNTIME_CONTEXT.detach(token)
  File "/Users/rchaves/Projects/langwatch-saas/langwatch/python-sdk/.venv/lib/python3.11/site-packages/opentelemetry/context/contextvars_context.py", line 53, in detach
    self._current_context.reset(token)
ValueError: <Token var=<ContextVar name='current_context' default={} at 0x1045fcb80> at 0x16d3a4800> was created in a different Context
Failed to detach context
Traceback (most recent call last):
  File "/Users/rchaves/Projects/langwatch-saas/langwatch/python-sdk/.venv/lib/python3.11/site-packages/pydantic/type_adapter.py", line 271, in _init_core_attrs
    self.core_schema = _getattr_no_parents(self._type, '__pydantic_core_schema__')
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/r